In [3]:
import pickle
import re
import pandas as pd

notes = pickle.load(open("pickle/notes.p", "rb" ))
links = pickle.load(open("pickle/links.p", "rb" ))

print(len(notes))
print(len(links))

75241
285557


In [10]:
note = notes[1341]
print(tune)


E2EB2B|efe^d3|edc BAG|A>cB/A/B3| E2EB2B|efe^d3|gfe ^cde|^c3B3|| b2b bag|f>ag/f/g3|efg fga|dgfg3| BAG cBA|B>FG/E/^D3E>FG/A/ BFG|F3E3|]


In [51]:
barsearch = re.compile(":?\|+:?\]?")
bars = [bar for bar in barsearch.split(tune) if bar!=""]
barlines = barsearch.findall(tune)
bars = [b+l for b, l in zip(bars,barlines)]

In [52]:
bars

['E2EB2B|',
 'efe^d3|',
 'edc BAG|',
 'A>cB/A/B3|',
 'E2EB2B|',
 'efe^d3|',
 'gfe ^cde|',
 '^c3B3||',
 'b2b bag|',
 'f>ag/f/g3|',
 'efg fga|',
 'dgfg3|',
 'BAG cBA|',
 'B>FG/E/^D3E>FG/A/ BFG|',
 'F3E3|]']

In [64]:
print(bars[-4:])
print(bars[:4])
print(bars[4:-4])

['dgfg3|', 'BAG cBA|', 'B>FG/E/^D3E>FG/A/ BFG|', 'F3E3|]']
['E2EB2B|', 'efe^d3|', 'edc BAG|', 'A>cB/A/B3|']
['E2EB2B|', 'efe^d3|', 'gfe ^cde|', '^c3B3||', 'b2b bag|', 'f>ag/f/g3|', 'efg fga|']


In [11]:

def process_abc_note(note):
    lines = note.strip().splitlines()
    title = [x for x in lines if re.compile("T:").match(x)]
    title = title[0].split(":")[1].strip() if len(title)>0 else None

    time = [x for x in lines if re.compile("M:").match(x)]
    time = time[0].split(":")[1].strip() if len(time)>0 else None

    length = [x for x in lines if re.compile("L:").match(x)]
    length = length[0].split(":")[1].strip() if len(length)>0 else None

    tune_type = [x for x in lines if re.compile("R:").match(x)]
    tune_type = tune_type[0].split(":")[1].strip() if len(tune_type)>0 else None

    keyindex = [i for i, x in enumerate(lines) if re.compile("K:").match(x)]
    key = lines[keyindex[0]].split(":")[1].strip() if len(keyindex)>0 else None
    keyindex = 0 if len(keyindex)==0 else keyindex[0]

    tune = [x for x in lines[keyindex:] if re.compile(".*\\|").match(x)]
    tune = "".join(tune) if len(tune)>0 else None
    
    barsearch = re.compile(":?\|+:?\]?")
    bars = [bar for bar in barsearch.split(tune) if bar!=""]
    barlines = barsearch.findall(tune)
    bars = [b+l for b, l in zip(bars,barlines)]

    features = {
        "title": title,
        "time": time,
        "length": length,
        "tune_type": tune_type,
        "key": key,
        "tune": tune,
        "bars": bars,
        "opening": bars[:4],
        "middle": bars[-4:],
        "ending": bars[-4:],
        "full_text": note
    }
    features["complete"] = None not in features.values()
    return(features)

In [85]:
features = [process_abc_note(note) for note in notes]
pickle.dump(features, open("features.p", "wb"))
len(features)

59881

In [103]:
complete_features = [x for x in features if x['complete']]

pickle.dump(complete_features, open("complete_features.p", "wb"))
len(complete_features)

30132

In [112]:
df_feat = pd.DataFrame(complete_features)

# Filter out long keys
df_feat = df_feat[df_feat['key'].map(lambda x: len(x)<10)]

df_feat.to_csv("features.csv")

PermissionError: [Errno 13] Permission denied: 'features.csv'

In [114]:
df_feat['key'].unique()

array(['C', 'G', 'G major', 'Ddor', 'D', 'A', 'Amix', 'Dmaj', 'Gmin', 'F',
       'Bmin', 'Gmix', 'D major', 'Gmaj', 'Amin', 'Em', 'D Major',
       'G Major', 'Amaj', 'Dm', 'Ador', 'Bb', 'C -8va', 'C bass', 'Edor',
       'Am', 'Emin', 'Dmin', 'Emaj', 'Gm', 'Fmaj', 'D minor', 'Gdor',
       'Bm', 'Dmix', 'F -8va', 'F bass', 'C major', 'None', 'Cmaj', 'Hp',
       'ADor', 'AMin', 'DMin', 'E', 'B minor', 'BMin', 'Eb', 'C Minor',
       'G Minor', 'Glyd', 'Bbmaj', 'GMaj', 'Ab', 'Cdor', 'Cm', 'Cmix',
       'Dphr', 'Fm', 'Emix', '', 'E minor', 'C minor', 'Bminor', 'EMin',
       'A Minor', 'Bdor', 'BDor', 'Fmin', 'DMix', 'C dorian', 'C Dor',
       'Ebmaj', 'A mix', 'B', 'GIon', 'F. CFG|', 'CMaj', 'Cmin',
       'G %1sharp', 'Am treble', 'DDor', 'DMaj', 'EMinor', 'Dphr ^F',
       'Dphr^F', 'Ephr^G', 'A=G', 'A % mix', 'E Dor', 'A Mix', 'B Minor',
       'E Minor', 'GDor', 'F#dor', 'G Dorian', 'F major', 'A minor',
       'Bmix', 'AMix=g', 'BMix=A', 'D maj', 'AMix', 'A_B^c', 'D min',
     

In [106]:
key = 'D "Bm" def ~B2 B d2 B | "Bm" def ~B2 B "A" ecA | "Bm" def ~B2 B Bcd | "A" c2 A Aaf ecA'
df_feat[df_feat['key']==key]

,complete,full_text,key,length,time,title,tune,tune_type
15339,True,\nX:1\nT:Brose and Butter.\nR:slip jig.\nS:Mil...,"D ""Bm"" def ~B2 B d2 B | ""Bm"" def ~B2 B ""A"" ecA...",1/8.,9/8.,Brose and Butter.,"K:D ""Bm"" def ~B2 B d2 B | ""Bm"" def ~B2 B ""A"" e...",slip jig.


In [115]:
df_feat

,complete,full_text,key,length,time,title,tune,tune_type
4,True,\nX:66\nT:066\nM:4/4\nL:1/8\nR:Reel\nZ:Grk-066...,C,1/8,4/4,066,G3A GE~E2|CE~E2 GAcd|eddc AGAc|dfec dcAG| EG~G...,Reel
5,True,\nI:abc-charset utf-8\n\nX:10112\nT:100 Psalm....,G,1/4,2/2,100 Psalm. JMP.102,"""No key signature shown"" G4 | G2F2 | E2D2 | G2...",.Psalm
6,True,\nX:102\nT:100 Psalm. JMP.102\nT:Old Hundredth...,G major,1/4,2/2,100 Psalm. JMP.102,"""No key signature shown"" G4 | G2F2 | E2D2 | G2...",.Psalm
7,True,\nX: 1\nT: 100 Watt Reels\nR: reel\nM: 4/4\nL:...,Ddor,1/8,4/4,100 Watt Reels,"[|: ""A"".d.DD/D/D .A.DD/D/D | .d.DD/D/D c2Bc | ...",reel
10,True,\nX: 12\nT: 10. FIRE IN THE MOUNTAIN\nB: Sam B...,D,1/8,C|,10. FIRE IN THE MOUNTAIN,F2A2 A2BA | F2A2 A2z2 | F2A2 A2BA | F2E2 E3D |...,reel
13,True,\nX:54\nT:10. Gaillarde\nT:Mille ducas\nC:Tiel...,Ddor,1/2,3/2,10. Gaillarde,"DDD|E2E|FFF|G2E|E>F G/E/|F2E|D^C/B,/^C|D2z:| |...",Galliard
19,True,\nX:1\nT: The 10 pound Fiddle\nC:J. Scott Skin...,A,1/8,4/4,The 10 pound Fiddle,|:E|A2 ED CE Af|eA fA GB BG|A2 EC Dc BA|GB eg ...,Reel
22,True,\nX: 1\nT: 10th Bat Crossing Rhine\nR: jig\nM:...,Amix,1/8,6/8,10th Bat Crossing Rhine,|: {g}A|{Gdc}d3/2f/2{g}A{Gdc}d2f|{g}f3/2g/2a{f...,jig
23,True,\nX: 2\nT: 10th Bat Crossing Rhine\nR: jig\nM:...,Amix,1/8,6/8,10th Bat Crossing Rhine,d>fA d2 f | f>ga f>ed | c2 e A>ce | e>fg f>ed ...,jig
24,True,\nX: 3\nT: 10th Bat Crossing Rhine\nR: jig\nM:...,Dmaj,1/8,6/8,10th Bat Crossing Rhine,d<fA d2 A | f>ga fed | c<eA c2 e | e>fg fed | ...,jig
